In [1]:
#!pip install fuzzywuzzy


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz, process

# Load files with appropriate encodings for each file
try:
    activity_consume_df = pd.read_csv('tn_activity_consume_his_활동소비내역_E.csv', encoding='utf-8-sig')
except UnicodeDecodeError:
    activity_consume_df = pd.read_csv('tn_activity_consume_his_활동소비내역_E.csv', encoding='cp949')

try:
    adv_consume_df = pd.read_csv('tn_adv_consume_his_사전소비내역_E.csv', encoding='utf-8-sig')
except UnicodeDecodeError:
    adv_consume_df = pd.read_csv('tn_adv_consume_his_사전소비내역_E.csv', encoding='cp949')


# 여행 정보 파일 (travel_df)
try:
    travel_df = pd.read_csv('tn_travel_여행_E.csv', encoding='utf-8-sig')
except UnicodeDecodeError:
    try:
        travel_df = pd.read_csv('tn_travel_여행_E.csv', encoding='cp949')
    except UnicodeDecodeError:
        travel_df = pd.read_csv('tn_travel_여행_E.csv', encoding='euc-kr')

# 방문지 정보 파일 (visit_area_info_df)
try:
    visit_area_info_df = pd.read_csv('tn_visit_area_info_방문지정보_E.csv', encoding='utf-8-sig')
except UnicodeDecodeError:
    try:
        visit_area_info_df = pd.read_csv('tn_visit_area_info_방문지정보_E.csv', encoding='cp949')
    except UnicodeDecodeError:
        visit_area_info_df = pd.read_csv('tn_visit_area_info_방문지정보_E.csv', encoding='euc-kr')


# 데이터 결합
adv_consume_travel_df = adv_consume_df.merge(travel_df, on="TRAVEL_ID", how="left")
adv_consume_combined_df = adv_consume_travel_df.merge(visit_area_info_df, on="TRAVEL_ID", how="left")
adv_consume_combined_df.to_csv('adv_consume_combined_df.csv', index=False, encoding='utf-8-sig')

3. 장소 이름(POI_NM) 정규화
동일한 브랜드의 다양한 지점을 하나로 묶기 위해 fuzzywuzzy 라이브러리를 활용하여 유사 이름을 기준으로 장소를 통합합니다.

In [3]:
# 주요 장소 이름 목록 정의 
base_poi_names = ["스타벅스", "이디야", "투썸플레이스", "더현대서울", "에버랜드", "롯데마트", "국립중앙박물관", "하나로마트", "롯데백화점", "CGV", "CU", "GS25"]

# 장소 이름을 정규화하는 함수
def normalize_poi_name(poi_name, base_names, threshold=80):
    if pd.isnull(poi_name):
        return poi_name
    match, score = process.extractOne(poi_name, base_names, scorer=fuzz.partial_ratio)
    return match if score >= threshold else poi_name

# 정규화 적용
adv_consume_combined_df['POI_NM'] = adv_consume_combined_df['POI_NM'].apply(lambda x: normalize_poi_name(x, base_poi_names))


4. 그룹화 및 분석
정규화된 POI_NM을 기준으로 평균 결제 금액과 방문 횟수를 계산하고, 방문 빈도가 높은 순서로 정렬하여 분석을 수행합니다.

In [4]:
# Group by normalized POI_NM and calculate average payment and visit count
poi_analysis_fuzzy_normalized = adv_consume_combined_df.groupby("POI_NM").agg(
    avg_payment=("PAYMENT_AMT_WON", "mean"),
    visit_count=("TRAVEL_ID", "count")
).sort_values(by="visit_count", ascending=False)

# Display or export the analysis results as needed
poi_analysis_fuzzy_normalized.head(20)


,avg_payment,visit_count
POI_NM,,
에버랜드,76693.194444,72
스타벅스,63445.375000,48
더현대서울,80417.021277,47
아주아파트,15289.800000,25
국립중앙박물관,62563.478261,23
캐리비안베이,41589.565217,23
2001아울렛 구로점,65791.250000,16
도원스타일 서울역점,113962.500000,16
센트럴시티터미널(호남),18996.000000,15


3. 여행 목적별 소비 분석

In [5]:
# 필요한 컬럼만 추출
data_for_analysis = adv_consume_combined_df[['TRAVEL_PURPOSE', 'POI_NM', 'PAYMENT_AMT_WON']].dropna()

# 여행 목적별 평균 및 총 소비 금액
purpose_analysis = data_for_analysis.groupby("TRAVEL_PURPOSE").agg(
    avg_payment=("PAYMENT_AMT_WON", "mean"),
    total_payment=("PAYMENT_AMT_WON", "sum"),
    visit_count=("POI_NM", "count")
).sort_values(by="total_payment", ascending=False)


4. 여행 목적 코드 해석을 위한 매핑 파일 로드 및 번역

In [7]:
# 코드 매핑 파일 불러오기
try:
    code_a_df = pd.read_csv('tc_codea_코드A.csv', encoding='cp949')
except UnicodeDecodeError:
    code_a_df = pd.read_csv('tc_codea_코드A.csv', encoding='euc-kr')

try:
    code_b_df = pd.read_csv('tc_codeb_코드B.csv', encoding='cp949')
except UnicodeDecodeError:
    code_b_df = pd.read_csv('tc_codeb_코드B.csv', encoding='euc-kr')

# 필요한 컬럼만 추출하고 코드 그룹 설명과 상세 코드 설명을 결합
code_a_df = code_a_df[['cd_a', 'cd_nm']]
code_b_df = code_b_df[['cd_a', 'cd_b', 'cd_nm']]
code_full_df = code_b_df.merge(code_a_df, on='cd_a', how='left')

# 코드 딕셔너리 생성
code_dict = code_full_df.set_index('cd_b')['cd_nm_x'].to_dict()

# TRAVEL_PURPOSE 코드 번역 함수
def translate_purpose(codes):
    code_list = str(codes).split(';')
    translated_list = [code_dict.get(code.strip(), code) for code in code_list]
    return "; ".join(translated_list)

# 번역 적용
purpose_analysis_translated = purpose_analysis.copy()
purpose_analysis_translated['Translated_TRAVEL_PURPOSE'] = purpose_analysis_translated.index.map(translate_purpose)

# 컬럼 순서 조정
purpose_analysis_translated = purpose_analysis_translated[['Translated_TRAVEL_PURPOSE', 'avg_payment', 'total_payment', 'visit_count']]


In [13]:
# 번역된 목적별 소비 분석 결과 확인
purpose_analysis_translated.head()


,Translated_TRAVEL_PURPOSE,avg_payment,total_payment,visit_count
TRAVEL_PURPOSE,,,,
1;23;27;4;6,"자연관광지; 사무실; 친환경 여행(플로깅 여행); 상업지구(거리, 시장, 쇼핑시설)...",400000.000000,7600000,19
1;2;3;4;6,"자연관광지; 역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등); 문화 시설(...",48984.586466,6514950,133
1;23;24;26;27,자연관광지; 사무실; 숙소; 인플루언서 따라하기 여행; 친환경 여행(플로깅 여행),480000.000000,5760000,12
2;22;27;4;6,"역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등); 친구/친지집; 친환경 여...",335200.000000,4357600,13
22;6;1;26;2,"친구/친지집; 테마시설(놀이공원, 워터파크); 자연관광지; 인플루언서 따라하기 여행...",313200.000000,3758400,12


In [11]:
import matplotlib.pyplot as plt

# Save the translated purpose analysis to a CSV file
purpose_analysis_translated.to_csv('Translated_Travel_Purpose_Analysis.csv', index=False, encoding='utf-8-sig')


In [15]:
translated_purpose_df = purpose_analysis_translated

# Initialize an empty set to store unique purposes
unique_purposes_set = set()

# Iterate over each row in the 'Translated_TRAVEL_PURPOSE' column
for purposes in translated_purpose_df['Translated_TRAVEL_PURPOSE'].dropna():
    # Split each entry by ';' and add to the set
    purpose_list = purposes.split(';')
    for purpose in purpose_list:
        unique_purposes_set.add(purpose.strip())

# Convert the set to a sorted list for readability
unique_purposes_list = sorted(unique_purposes_set)

# Display all unique purposes
unique_purposes_list


['기타',
 '등반 여행',
 '레저/스포츠 관련 시설(스키, 카트, 수상레저)',
 '문화 시설(공연장, 영화관, 전시관 등)',
 '반려동물 동반 여행',
 '사무실',
 '산책로, 둘레길 등',
 '상업지구(거리, 시장, 쇼핑시설)',
 '상점',
 '숙소',
 '식당/카페',
 '역, 터미널, 고속도로 휴게소',
 '역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등)',
 '인플루언서 따라하기 여행',
 '자연관광지',
 '지역 축제/행사',
 '집',
 '체험 활동 관광지',
 '친구/친지집',
 '친환경 여행(플로깅 여행)',
 '테마시설(놀이공원, 워터파크)']

In [2]:
import pandas as pd

# Load the Translated_Travel_Purpose_Analysis.csv file
try:
    translated_purpose_df = pd.read_csv('Translated_Travel_Purpose_Analysis.csv', encoding='utf-8-sig')
except UnicodeDecodeError:
    translated_purpose_df = pd.read_csv('Translated_Travel_Purpose_Analysis.csv', encoding='cp949')

# Initialize a dictionary to count occurrences of each purpose
purpose_count = {}

# Iterate over each row in the 'Translated_TRAVEL_PURPOSE' column
for purposes in translated_purpose_df['Translated_TRAVEL_PURPOSE'].dropna():
    # Split each entry by ';' and count each purpose
    purpose_list = purposes.split(';')
    for purpose in purpose_list:
        purpose = purpose.strip()
        if purpose in purpose_count:
            purpose_count[purpose] += 1
        else:
            purpose_count[purpose] = 1

# Calculate total occurrences of all purposes
total_purposes = sum(purpose_count.values())

# Calculate the percentage for each purpose
purpose_percentage = {purpose: (count / total_purposes) * 100 for purpose, count in purpose_count.items()}

# Convert to DataFrame for better readability and sort by percentage
purpose_percentage_df = pd.DataFrame(list(purpose_percentage.items()), columns=['Purpose', 'Percentage']).sort_values(by='Percentage', ascending=False)

# Display the DataFrame
purpose_percentage_df


,Purpose,Percentage
9,친구/친지집,15.204678
0,자연관광지,10.350877
4,"테마시설(놀이공원, 워터파크)",9.064327
5,"역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등)",8.771930
3,"상업지구(거리, 시장, 쇼핑시설)",6.783626
11,집,6.666667
8,인플루언서 따라하기 여행,5.964912
10,"레저/스포츠 관련 시설(스키, 카트, 수상레저)",5.146199
7,숙소,4.502924
6,"문화 시설(공연장, 영화관, 전시관 등)",4.093567
